# Uppgift 8, Kapitel 10: Skapa en Chattbot med RAG

## Uppgift
Skapa en chattbot som svarar på frågor utifrån ett dokument. Du ska använda RAG (Retrieval Augmented Generation) för att:
1. Läsa in ett PDF-dokument
2. Dela upp dokumentet i chunks
3. Skapa embeddings för varje chunk
4. Implementera semantisk sökning
5. Generera svar utifrån relevant kontext

Vi använder boken "Lär dig AI från grunden - Tillämpad maskininlärning med Python" som källdokument.

## 1. Installation och Import av bibliotek

In [3]:
# Installera nödvändiga paket
# pip install google-genai pypdf numpy

In [4]:
import numpy as np
from google import genai
from pypdf import PdfReader
import os

## 2. Ställ in API-nyckel och klient

**VIKTIGT:** Du behöver skapa en egen API-nyckel från https://aistudio.google.com/
- Gå till "Create API key"
- Du behöver registrera ett bankkort för gratis provperiod
- DELA ALDRIG din API-nyckel offentligt!

För säkerhet kan du använda miljövariabler: `api_key = os.getenv('API_KEY')`

In [ ]:
# Ersätt denna med din egen API-nyckel
api_key = os.getenv('GOOGLE_API_KEY')  # Hämta från miljövariabler för säkerhet
if not api_key:
    api_key = ''  # Eller ange manuellt
client = genai.Client(api_key=api_key)
model = "gemini-2.0-flash"

## 3. Läs in PDF-dokumentet

In [6]:
# Läs PDF-filen
pdf_path = "publicerad_bok 4.pdf"

try:
    reader = PdfReader(pdf_path)
    
    # Extrahera all text från PDF:en
    text = ""
    for i, page in enumerate(reader.pages):
        try:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        except Exception as e:
            print(f"Varning: Kunde inte extrahera text från sida {i+1}: {e}")
            continue
    
    if not text:
        raise ValueError("Ingen text kunde extraheras från PDF:en. PDF:en kan vara krypterad eller skadad.")
    
    print(f"✓ PDF inläst framgångsrikt")
    print(f"  Total textlängd: {len(text)} tecken")
    print(f"  Antal sidor: {len(reader.pages)}")
    
except FileNotFoundError:
    print(f"❌ Fel: Filen '{pdf_path}' hittades inte")
    print(f"   Kontrollera att filen finns i samma mapp som notebooken")
    raise
except Exception as e:
    print(f"❌ Fel vid läsning av PDF: {e}")
    raise

✓ PDF inläst framgångsrikt
  Total textlängd: 443311 tecken
  Antal sidor: 348


In [7]:
# Visa ett utdrag från dokumentet
print(text[0:500])

LÄR DIG AI FRÅN
GRUNDEN - TILLÄMPAD
MASKININLÄRNING MED
PYTHON
Antonio Prgomet
Terese Johnson
Amanda Solberg
Linus Rundberg StreuliDetta verk är skyddat av upphovsrättslagen.
Den som bryter mot upphovsrättslagen kan åtalas av allmän åklagare och dömas till bö-
ter eller fängelse i upp till två år samt bli skyldig att erlägga ersättning till upphovsman
eller rättsinnehavare.
© Pedagogicus PublishingInnehållsförteckning
Förord 7
Bokens hemsida . . . . . . . . . . . . . . . . . . . . . . . . . . . 


## 4. Chunking - Dela upp texten i mindre delar

Vi använder **fixed-length chunking** med överlappning för att säkerställa att viktiga koncept inte splittras mellan chunks.

In [8]:
def create_chunks(text, chunk_size=1000, overlap=200):
    """
    Dela upp text i chunks med överlappning.
    
    Args:
        text: Texten att dela upp
        chunk_size: Storlek på varje chunk (antal tecken)
        overlap: Överlappning mellan chunks (antal tecken)
    
    Returns:
        Lista med chunks
    """
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

# Skapa chunks
chunks = create_chunks(text, chunk_size=1000, overlap=200)
print(f"Antal chunks skapade: {len(chunks)}")

Antal chunks skapade: 555


In [9]:
# Visa första chunken
print("Första chunk:")
print(chunks[0])

Första chunk:
LÄR DIG AI FRÅN
GRUNDEN - TILLÄMPAD
MASKININLÄRNING MED
PYTHON
Antonio Prgomet
Terese Johnson
Amanda Solberg
Linus Rundberg StreuliDetta verk är skyddat av upphovsrättslagen.
Den som bryter mot upphovsrättslagen kan åtalas av allmän åklagare och dömas till bö-
ter eller fängelse i upp till två år samt bli skyldig att erlägga ersättning till upphovsman
eller rättsinnehavare.
© Pedagogicus PublishingInnehållsförteckning
Förord 7
Bokens hemsida . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7
Bokens målgrupp . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7
Element i boken . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7
Språk . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 8
Bokens upplägg . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 8
Lycka till . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 9
I Introduktion till maskininlärn

## 5. Embeddings - Konvertera text till vektorer

Embeddings representerar text som numeriska vektorer. Semantiskt lika texter får närliggande vektorer.
Se kapitel 10 i kursboken för mer information.

In [11]:
from google.genai import types
import time

def create_embeddings(text_list, model="text-embedding-004", batch_size=100):
    """
    Skapa embeddings för en lista av texter (med batch-processing).
    
    Args:
        text_list: Lista av texter eller en enskild text
        model: Embedding-modell att använda
        batch_size: Antal texter per batch (standar API-gräns: 100)
    
    Returns:
        Lista av embedding-vektorer
    """
    if isinstance(text_list, str):
        text_list = [text_list]
    
    all_embeddings = []
    total_batches = (len(text_list) + batch_size - 1) // batch_size
    
    print(f"Skapar embeddings för {len(text_list)} chunks i {total_batches} batches...")
    
    for batch_idx in range(0, len(text_list), batch_size):
        batch = text_list[batch_idx:batch_idx + batch_size]
        current_batch_num = (batch_idx // batch_size) + 1
        
        try:
            print(f"  Batch {current_batch_num}/{total_batches}...", end=" ", flush=True)
            
            response = client.models.embed_content(
                model=model,
                contents=batch,
                config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY")
            )
            
            for emb in response.embeddings:
                all_embeddings.append(emb.values)
            
            print("✓")
            
            # Liten paus mellan batches för att undvika rate limiting
            if current_batch_num < total_batches:
                time.sleep(0.5)
                
        except Exception as e:
            print(f"❌ Fel i batch {current_batch_num}: {e}")
            raise
    
    print(f"\n✓ Alla embeddings skapade!")
    return all_embeddings

# Skapa embeddings för alla chunks
embeddings_list = create_embeddings(chunks, batch_size=100)

print(f"Antal embeddings skapade: {len(embeddings_list)}")
print(f"Storlek på varje embedding: {len(embeddings_list[0])}")

Skapar embeddings för 555 chunks i 6 batches...
  Batch 1/6... 

❌ Fel i batch 1: 403 PERMISSION_DENIED. {'error': {'code': 403, 'message': 'Your API key was reported as leaked. Please use another API key.', 'status': 'PERMISSION_DENIED'}}


ClientError: 403 PERMISSION_DENIED. {'error': {'code': 403, 'message': 'Your API key was reported as leaked. Please use another API key.', 'status': 'PERMISSION_DENIED'}}

In [ ]:
# Visa första 10 värden från första embeddings-vektorn
print(f"Första embedding (första 10 värden):")
print(embeddings_list[0][0:10])

Första embedding (första 10 värden):
[-0.09116102, 0.03508013, -0.038605068, 0.03724536, 0.07020524, -8.3170125e-05, -0.013250188, 0.01706417, 0.039274067, -0.019324668]


## 6. Semantisk sökning - Cosine Similarity

Vi använder cosinuslikhet för att mäta hur lik en användarfråga är jämfört med varje chunk.

In [ ]:
def cosine_similarity(vec1, vec2):
    """
    Beräkna cosinuslikhet mellan två vektorer.
    Värde mellan -1 och 1, där 1 betyder identisk riktning.
    
    Säkrare version som hanterar edge cases.
    """
    try:
        dot_product = np.dot(vec1, vec2)
        norm_vec1 = np.linalg.norm(vec1)
        norm_vec2 = np.linalg.norm(vec2)
        
        # Undvik division med noll
        if norm_vec1 == 0 or norm_vec2 == 0:
            return 0.0
        
        return float(dot_product / (norm_vec1 * norm_vec2))
    except Exception as e:
        print(f"Fel vid cosinusberäkning: {e}")
        return 0.0

def semantic_search(query, chunks, embeddings_list, k=5):
    """
    Sök efter de k mest relevanta chunks för en given fråga.
    
    Args:
        query: Användarens fråga
        chunks: Lista av textchunks
        embeddings_list: Lista av embedding-vektorer
        k: Antal toppta chunks att returnera
    
    Returns:
        Lista med de k mest relevanta chunks
    """
    print(f"🔍 Söker efter relevanta delar för: '{query}'")
    
    try:
        # Skapa embedding för frågan
        print("  - Genererar frågeembedding...", end=" ", flush=True)
        query_embeddings = create_embeddings(query, batch_size=1)
        query_embedding = query_embeddings[0]
        print("✓")
        
        # Beräkna likhet mellan fråga och alla chunks
        print(f"  - Jämför med {len(embeddings_list)} chunks...", end=" ", flush=True)
        similarity_scores = []
        
        for i, chunk_embedding in enumerate(embeddings_list):
            similarity_score = cosine_similarity(query_embedding, chunk_embedding)
            similarity_scores.append((i, similarity_score))
        
        print("✓")
        
        # Sortera efter likhet (högst först)
        similarity_scores.sort(key=lambda x: x[1], reverse=True)
        
        # Returnera top k chunks
        top_indices = [index for index, score in similarity_scores[:k]]
        top_scores = [score for index, score in similarity_scores[:k]]
        
        print(f"  - Hittade {len(top_indices)} relevanta chunks")
        for idx, (chunk_idx, score) in enumerate(zip(top_indices, top_scores), 1):
            print(f"    {idx}. Likhet: {score:.3f}")
        
        result_chunks = [chunks[index] for index in top_indices]
        return result_chunks
        
    except Exception as e:
        print(f"❌ Fel i semantisk sökning: {e}")
        raise

# Testa semantisk sökning
print("Testar semantisk sökning...\n")
test_query = "Vad är maskininlärning?"
try:
    relevant_chunks = semantic_search(test_query, chunks, embeddings_list, k=3)
    print(f"\n✓ Första relevanta chunk (första 300 tecken):\n{relevant_chunks[0][:300]}...")
except Exception as e:
    print(f"Kunde inte genomföra sökning: {e}")

Skapar embeddings för 1 chunks i 1 batches...
  Batch 1/1... ✓

✓ Alla embeddings skapade!
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\SebbePwnYou\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3699, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SebbePwnYou\AppData\Local\Temp\ipykernel_168\4070036183.py", line 43, in <module>
    relevant_chunks = semantic_search(test_query, chunks, embeddings_list, k=3)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SebbePwnYou\AppData\Local\Temp\ipykernel_168\4070036183.py", line 26, in semantic_search
    query_embedding = query_embedding_response.embeddings[0].values
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'list' object has no attribute 'embeddings'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\SebbePwnYou\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 2194, in showtraceback
    stb = self.Inter

## 7. RAG - Generera svar med kontext

Nu kombinerar vi retrieval (sökning) med generation (svar) för att skapa en chattbot.

In [ ]:
# Systemmeddelande för chattbotten
system_prompt = """Du är en hjälpsam AI-assistent som svarar på frågor baserat ENDAST på 
den kontexten som tillhandahålls. Du är en expert på boken 'Lär dig AI från grunden - Tillämpad 
maskininlärning med Python'.

Instruktioner:
1. Svara ENDAST baserat på kontexten som ges
2. Om det inte finns tillräcklig information i kontexten, säg: "Det vet jag inte baserat på dokumentet"
3. Försök INTE att gissa eller lägga till information utanför kontexten
4. Formulera dig enkelt och dela upp svaret i stycken
5. Citera relevanta delar från dokumentet när lämpligt"""

def generate_user_prompt(query, context_chunks):
    """
    Skapa ett användarmeddelande med fråga och kontext.
    """
    context = "\n\n---\n\n".join(context_chunks)
    user_prompt = f"""Baserat på följande kontext från dokumentet, svara på denna fråga:

KONTEXT:
{context}

FRÅGA: {query}

SVAR:"""
    return user_prompt

def generate_response(query, model=model, k=5):
    """
    Generera ett svar på en fråga med RAG.
    
    Args:
        query: Användarens fråga
        model: LLM-modell att använda
        k: Antal chunks att använda som kontext
    
    Returns:
        Svar från modellen
    """
    # Hämta relevanta chunks
    context_chunks = semantic_search(query, chunks, embeddings_list, k=k)
    
    # Skapa prompt
    user_message = generate_user_prompt(query, context_chunks)
    
    # Generera svar
    response = client.models.generate_content(
        model=model,
        config=genai.types.GenerateContentConfig(
            system_instruction=system_prompt
        ),
        contents=user_message
    )
    return response.text

print("RAG-funktioner är klara!")

RAG-funktioner är klara!


## 8. Testa chattbotten

Testa med några exempelfrågor relaterade till boken.

In [ ]:
# Testa fråga 1
fråga1 = "Vad är prompt engineering?"
print(f"\n{'='*60}")
print(f"FRÅGA: {fråga1}")
print(f"{'='*60}")
svar1 = generate_response(fråga1, k=3)
print(svar1)


FRÅGA: Vad är prompt engineering?
Prompt engineering handlar om att ställa "effektiva frågor" till chattbottar för att få "bättre" svar. Som tumregel är det bra att vara så specifik, deskriptiv och detaljerad som möjligt om önskad kontext, utfall, längd, format och stil.


In [ ]:
# Testa fråga 2
fråga2 = "Vad är RAG?"
print(f"\n{'='*60}")
print(f"FRÅGA: {fråga2}")
print(f"{'='*60}")
svar2 = generate_response(fråga2, k=3)
print(svar2)


FRÅGA: Vad är RAG?
En RAG-modell innehåller två delar. Den första delen är en retriever, som söker efter relevanta stycken i en större text. Dessa stycken skickas sedan vidare som kontext till den andra delen, en generator som genererar svaren utifrån den givna kontexten.


In [ ]:
# Testa fråga 3
fråga3 = "Vad är chunking och embeddings?"
print(f"\n{'='*60}")
print(f"FRÅGA: {fråga3}")
print(f"{'='*60}")
svar3 = generate_response(fråga3, k=3)
print(svar3)


FRÅGA: Vad är Chunking?
Chunking kan göras utan hänsyn till innehållet i texten, vilket kan leda till förlust av information och sämre semantiska sökningar.

Semantic chunking är en metod för att dela upp text i mindre delar baserat på den semantiska betydelsen och innebär att skapa delar som innehåller meningar som ligger nära varandra i betydelse. Metoden innefattar sentence-based chunking, följt av skapandet av embeddings av meningarna och en semantisk sökning för att jämföra deras innehåll. Resultaten används sedan för att skapa chunks som innehåller meningar som är semantiskt relaterade.


In [ ]:
# Testa en fråga som inte är relaterad till dokumentet
fråga4 = "Förklara översiktligt hur man kan evaluera en chattbot."
print(f"\n{'='*60}")
print(f"FRÅGA: {fråga4}")
print(f"{'='*60}")
svar4 = generate_response(fråga4, k=3)
print(svar4)


FRÅGA: Vad är favoritkolorerna på stjärnorna?
Det vet jag inte baserat på dokumentet.


## 9. Interaktiv chattbot

En enkel interaktiv chattbot där du kan ställa egna frågor. Skriv "quit" för att avsluta.

In [ ]:
print("\n" + "="*60)
print("ENKLA TESTFRÅGOR - Visa/dölj svar nedan")
print("="*60)

# Fördefinierade testfrågor
test_questions = [
    "Vad är maskininlärning?",
    "Vilka är huvudtyperna av maskininlärning?",
    "Vad är Python?",
    "Vad är supervised learning?",
    "Vad är unsupervised learning?",
]

# Ställ dina egna frågor genom att ändra denna lista:
user_questions = [
    "Vad är Deep Learning?",
    "Nämn några populära Python-bibliotek för ML",
]

all_questions = test_questions + user_questions

print(f"\nKörs {len(all_questions)} testfrågor...\n")

results = []
for idx, question in enumerate(all_questions, 1):
    print(f"[{idx}/{len(all_questions)}] Fråga: {question}")
    try:
        answer = generate_response(question, k=3)
        results.append({"question": question, "answer": answer})
        print(f"Svar: {answer[:200]}...\n")
    except Exception as e:
        print(f"Fel: {e}\n")

print("="*60)
print(f"✓ Alla {len(results)} frågor är klara!")


INTERAKTIV CHATTBOT - Baserad på 'Lär dig AI från grunden'
Ställ en fråga och chattbotten svarar baserat på boken.
Skriv 'quit' eller 'exit' för att sluta.



## 10. Evaluering av chattbotten (Bonus)

Vi kan evaluera kvaliteten på svaren genom att jämföra med ideala svar.

In [ ]:
# Testdata för evaluering
test_data = [
    {
        "question": "Vad är supervised learning?",
        "ideal_answer": "Supervised learning är en typ av maskininlärning där modellen tränas på märkta exempel."
    },
    {
        "question": "Nämn två populära Python-bibliotek för maskininlärning.",
        "ideal_answer": "TensorFlow och scikit-learn är två populära Python-bibliotek för maskininlärning."
    }
]

print("Test-dataset laddat med 2 testfrågor.")
print(test_data[0])

Test-dataset laddat med 2 testfrågor.
{'question': 'Vad är supervised learning?', 'ideal_answer': 'Supervised learning är en typ av maskininlärning där modellen tränas på märkta exempel.'}


In [ ]:
# Systemmeddelande för evaluering
evaluation_system_prompt = """Du är ett intelligentakterar som utvärderar svar från en AI-assistent.
Bedöm svaret enligt följande skala:
- 1.0: Svaret är mycket bra och nära det ideala svaret
- 0.5: Svaret är delvis korrekt men saknar detaljer
- 0.0: Svaret är felaktigt eller inte relaterat

Ge ett tal mellan 0 och 1 tillsammans med en kort förklaring."""

def evaluate_response(question, ai_answer, ideal_answer):
    """
    Evaluera ett svar från chattbotten.
    """
    evaluation_prompt = f"""Fråga: {question}
AI-assistentens svar: {ai_answer}
Idealiskt svar: {ideal_answer}

Utvärdera AI-svaret:"""
    
    response = client.models.generate_content(
        model=model,
        config=genai.types.GenerateContentConfig(
            system_instruction=evaluation_system_prompt
        ),
        contents=evaluation_prompt
    )
    return response.text

# Evaluera första testfråga
test_question = test_data[0]["question"]
ideal_answer = test_data[0]["ideal_answer"]
ai_answer = generate_response(test_question, k=3)

print(f"\nFÅGA: {test_question}")
print(f"\nAI-SVAR:\n{ai_answer}")
print(f"\nIDEALT SVAR:\n{ideal_answer}")
print(f"\nEVALUERING:\n{evaluate_response(test_question, ai_answer, ideal_answer)}")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\SebbePwnYou\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3699, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SebbePwnYou\AppData\Local\Temp\ipykernel_168\3682832112.py", line 32, in <module>
    ai_answer = generate_response(test_question, k=3)
                ^^^^^^^^^^^^^^^^^
NameError: name 'generate_response' is not defined. Did you mean: 'evaluate_response'?

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\SebbePwnYou\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 2194, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SebbePwnYou\anaconda3\Lib\site-packages\IPython\core\ultratb.py", line 1188, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^

## Sammanfattning

Du har nu skapat en fullständig RAG-baserad chattbot som:

1. **Läser PDF-dokument** - Extraherar text från PDF-filer
2. **Chunkar text** - Delar upp text i hanterbar storlek
3. **Skapar embeddings** - Konverterar text till vektorer
4. **Söker semantiskt** - Hittar relevanta delar baserat på likhet
5. **Genererar svar** - Använder LLM för att skapa sammanhängande svar
6. **Evaluerar resultat** - Bedömer kvaliteten på svaren

### Vidare fördjupning:
- Utforska LangChain: https://academy.langchain.com/
- Lär dig om olika embedding-modeller
- Experimentera med olika chunk-storlekar
- Implementera minneshantering för flerturs-konversationer